# 卷积

## 1. 一维卷积

In [62]:
import torch
from torch import Tensor, nn
import torch.nn.functional as F

conv1 = nn.Conv1d(10, 5, kernel_size=1, padding=0, bias=False)
nn.init.constant_(conv1.weight, 1)
conv1.weight =nn.Parameter(torch.arange(10.0, dtype=torch.float).reshape(1, 10, 1).expand(5, 10, 1))
print('conv1', conv1.weight.shape)

x = torch.arange(10.0, dtype=torch.float)

x1 = torch.reshape(x, (1, 10, 1))
y1 = conv1(x1)

print('x1', x1.shape, x1)
print('y1', y1.shape, y1)

def conv_1d(input: Tensor, weight: Tensor):
    # input = (batch, input_channels, iW)
    # weight = (output_channels, input_channels, iW)
    o = torch.zeros(input.shape[0], weight.shape[0], input.shape[2])  # (batch, output_channels, iW) = 1, 5, 1
    
    for batch in range(o.shape[0]): # batch
        for c in range(o.shape[1]): # output_channels
            o[batch, c] = (weight[c] * input[batch]).sum() # 求和 s += weight[c][j][iW] * input[batch][j][iW]
    
    return o    


y1_ = conv_1d(x1, conv1.weight)
print('y1_', y1_.shape, y1_)

conv1 torch.Size([5, 10, 1])
x1 torch.Size([1, 10, 1]) tensor([[[0.],
         [1.],
         [2.],
         [3.],
         [4.],
         [5.],
         [6.],
         [7.],
         [8.],
         [9.]]])
y1 torch.Size([1, 5, 1]) tensor([[[285.],
         [285.],
         [285.],
         [285.],
         [285.]]], grad_fn=<ConvolutionBackward0>)
y1_ torch.Size([1, 5, 1]) tensor([[[285.],
         [285.],
         [285.],
         [285.],
         [285.]]], grad_fn=<CopySlices>)


## 2. 二维卷积

In [61]:
conv2 = nn.Conv2d(10, 5, kernel_size=1, padding=0, bias=False)
nn.init.constant_(conv2.weight, 1)
print('conv2', conv2.weight.shape)

x = torch.arange(60.0, dtype=torch.float)

x2 = torch.reshape(x, (1, 10, 2, 3))
print('x2', x2.shape, x2)

y2 = conv2(x2)
print('y2', y2.shape, y2)

def conv_2d(input: Tensor, weight: Tensor):
    # input = (batch, input_channels, iH, iW)
    # weight = (output_channels, input_channels, kH, kW)
    (_, _, kH, kW) = weight.shape
    (_, _, iH, iW) = input.shape
    o = torch.zeros(input.shape[0], weight.shape[0], iH - kH + 1, iW - kH + 1)  # (batch, output_channels, iH - kH + 1, iW - kH + 1) = 1, 5, 1, 1
    
    for batch in range(o.shape[0]): # batch
        for oc in range(o.shape[1]): # output_channels
            for ic in range(input.shape[1]):
                for i in range(o.shape[2]):
                    for j in range(o.shape[3]):
                        o[batch, oc, i, j] += (weight[oc, ic, :, :] * input[batch, ic, i:i+kH, j:j+kW]).sum()    
    return o    


y2_ = conv_2d(x2, conv2.weight)
print('y2_', y2_.shape, y2_)

conv2 torch.Size([5, 10, 1, 1])
x2 torch.Size([1, 10, 2, 3]) tensor([[[[ 0.,  1.,  2.],
          [ 3.,  4.,  5.]],

         [[ 6.,  7.,  8.],
          [ 9., 10., 11.]],

         [[12., 13., 14.],
          [15., 16., 17.]],

         [[18., 19., 20.],
          [21., 22., 23.]],

         [[24., 25., 26.],
          [27., 28., 29.]],

         [[30., 31., 32.],
          [33., 34., 35.]],

         [[36., 37., 38.],
          [39., 40., 41.]],

         [[42., 43., 44.],
          [45., 46., 47.]],

         [[48., 49., 50.],
          [51., 52., 53.]],

         [[54., 55., 56.],
          [57., 58., 59.]]]])
y2 torch.Size([1, 5, 2, 3]) tensor([[[[270., 280., 290.],
          [300., 310., 320.]],

         [[270., 280., 290.],
          [300., 310., 320.]],

         [[270., 280., 290.],
          [300., 310., 320.]],

         [[270., 280., 290.],
          [300., 310., 320.]],

         [[270., 280., 290.],
          [300., 310., 320.]]]], grad_fn=<ConvolutionBackward0>)
y2_ tor